In [2]:
!pip install open3d pyntcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 408.6 MB 27 kB/s 
     |████████████████████████████████| 346 kB 53.2 MB/s 
     |████████████████████████████████| 8.8 MB 45.1 MB/s 
     |████████████████████████████████| 3.1 MB 28.5 MB/s 
     |████████████████████████████████| 1.1 MB 51.2 MB/s 
     |████████████████████████████████| 596 kB 48.8 MB/s 
     |████████████████████████████████| 428 kB 38.7 MB/s 
     |████████████████████████████████| 344 kB 16.4 MB/s 
     |████████████████████████████████| 54 kB 561 kB/s 
     |████████████████████████████████| 1.2 MB 46.7 MB/s 
     |████████████████████████████████| 80 kB 6.7 MB/s 
     |████████████████████████████████| 54 kB 1.9 MB/s 
     |████████████████████████████████| 132 kB 45.4 MB/s 
     |████████████████████████████████| 561 kB 54.2 MB/s 
     |████████████████████████████████| 133 kB 47.1 MB/s 
     |███████████████████████████████

In [1]:
import open3d as o3d
import os
path = "/content/original.ply"
import numpy as np
from pyntcloud import PyntCloud

In [2]:
# Reading the point cloud using PCD
pcd_original = o3d.io.read_point_cloud(path)
print(pcd_original)

PointCloud with 1089091 points.


In [3]:
pcd_xyz = np.asarray(pcd_original.points)
pcd_rgb = np.asarray(pcd_original.colors)

In [4]:
full_pc = np.concatenate((pcd_xyz,pcd_rgb),axis=1)

In [8]:
original_indexes = range(0,len(full_pc))
original_indexes

range(0, 1089091)

In [9]:
import time
original_indexes= list(original_indexes)
patch_pcs =[]
iteration=0
start_time = time.time()
while original_indexes:
    dists = ((full_pc[original_indexes[0]][0:3] - full_pc[original_indexes,0:3])**2).sum(-1) # Calculating distances relative to all the remaining points
    patch = sorted(enumerate(dists), key=lambda i: i[1]) # (index , distance) sorted by distance
    if iteration%100==0:
        print("Reached iteration : ",iteration)
        print("Points remaining : ",len(patch))
        print("--- %s seconds ---" % (time.time() - start_time))
    iteration+=1 
    if(len(patch)>=2048):
        patch =patch[0:2048] # Taking the closest 2048 points
    else:
        patch =patch[0:] # In case we reach the last block and it has less then 2048 points
    patch_pc=[] # Placeholder for a single patch with exact points
    for p in patch: # for each index in the patch
        patch_pc.append(full_pc[p[0]].tolist()) # add the real points to patch_pc 
        original_indexes.remove(p[0]) # remove the index from the original indexes
    original_indexes=list(range(0,len(original_indexes)))
    patch_pcs.append(patch_pc)

Reached iteration :  0
Points remaining :  1089091
--- 1.062692642211914 seconds ---
Reached iteration :  100
Points remaining :  884291
--- 288.9132297039032 seconds ---
Reached iteration :  200
Points remaining :  679491
--- 518.1095476150513 seconds ---
Reached iteration :  300
Points remaining :  474691
--- 702.0591230392456 seconds ---
Reached iteration :  400
Points remaining :  269891
--- 841.0683588981628 seconds ---
Reached iteration :  500
Points remaining :  65091
--- 938.0831229686737 seconds ---


In [31]:
patch_pcs[-1][-1]

[118.0,
 47.0,
 63.0,
 0.35294117647058826,
 0.3411764705882353,
 0.33725490196078434]

In [38]:
numberp = 2048 - len(patch_pcs[-1])
numberp

0

In [32]:
for i in range(0,445):
  patch_pcs[-1].append(patch_pcs[-1][-1])

In [33]:
len(patch_pcs[-1])

2048

In [37]:
import numpy as np
np_patches = np.asarray(patch_pcs)

In [39]:
pcdexp = o3d.geometry.PointCloud()
pcdexp.points = o3d.utility.Vector3dVector(np_patches[1][:,0:3])
pcdexp.colors = o3d.utility.Vector3dVector(np_patches[1][:,3:6])
o3d.io.write_point_cloud("exp_patch.ply", pcdexp)

True

In [41]:
for i in range(0,len(np_patches)):
    pc = o3d.geometry.PointCloud() #Creating the pointcloud
    pc.points= o3d.utility.Vector3dVector(np_patches[i][:,0:3]) # Setting geometry
    pc.colors= o3d.utility.Vector3dVector(np_patches[i][:,3:6]) # Setting attributes
    name= "solider_patches/patch_"+str(i)+".ply"
    o3d.io.write_point_cloud(name, pc)

In [42]:
!zip -r /content/solider_patches.zip /content/solider_patches


  adding: content/solider_patches/ (stored 0%)
  adding: content/solider_patches/patch_71.ply (deflated 78%)
  adding: content/solider_patches/patch_509.ply (deflated 78%)
  adding: content/solider_patches/patch_236.ply (deflated 78%)
  adding: content/solider_patches/patch_414.ply (deflated 78%)
  adding: content/solider_patches/patch_400.ply (deflated 78%)
  adding: content/solider_patches/patch_471.ply (deflated 78%)
  adding: content/solider_patches/patch_324.ply (deflated 78%)
  adding: content/solider_patches/patch_32.ply (deflated 78%)
  adding: content/solider_patches/patch_259.ply (deflated 78%)
  adding: content/solider_patches/patch_40.ply (deflated 78%)
  adding: content/solider_patches/patch_355.ply (deflated 78%)
  adding: content/solider_patches/patch_231.ply (deflated 78%)
  adding: content/solider_patches/patch_429.ply (deflated 78%)
  adding: content/solider_patches/patch_126.ply (deflated 78%)
  adding: content/solider_patches/patch_458.ply (deflated 78%)
  adding: c